In [196]:
import glob
import os
import warnings
import mlflow
import mlflow.pytorch 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split
from torchvision import transforms
from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
importlib.reload(help_files._1_visuals_script)
# importlib.reload(help_files._01_load_data)
import  help_files._1_visuals_script  as pauls_vs
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [197]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [198]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")


DataFrames have been loaded successfully.


In [199]:
# # end sample or small sample    
if whole_data_set == True:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=300, random_state=RSEED)
    test_df = test_df.sample(n=300, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

Using the whole data set


Calss definition dataloader (do not change over models)

In [200]:
import pydicom
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset
import numpy as np

class MRILocalizationDataset(Dataset):
    def __init__(self, data, transform=None, has_coordinates=True):
        """
        Args:
            data (pd.DataFrame): DataFrame containing image paths and other info.
            transform (callable, optional): Optional transform to be applied on a sample.
            has_coordinates (bool, optional): Whether the dataset includes x, y coordinates.
        """
        self.data = data
        self.transform = transform
        self.has_coordinates = has_coordinates

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Get the image path
        img_path = self.data.iloc[idx]['image_path']
        
        # Read the DICOM file
        dicom_image = pydicom.dcmread(img_path)
        
        # Convert the DICOM pixel data to a NumPy array
        image_array = dicom_image.pixel_array
        
        # Convert NumPy array to PIL Image
        image = Image.fromarray(image_array)
        
        # Convert to RGB if necessary (some DICOM files might be grayscale)
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Apply transformations (e.g., resizing, normalization)
        if self.transform:
            image = self.transform(image)
        
        # Return image with or without coordinates
        if self.has_coordinates:
            x = torch.tensor(self.data.iloc[idx]['x'], dtype=torch.float32)
            y = torch.tensor(self.data.iloc[idx]['y'], dtype=torch.float32)
            return image, torch.tensor([x, y])
        else:
            return image





training

In [201]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
# %pip install keras
# %pip install tensorflow
# from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping
import numpy as np  # Import numpy for setting the random seed

# Define the transformation to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the expected input size of the model
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Create the dataset
dataset = MRILocalizationDataset(data=train_df, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 64
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load ResNet-50 and set up for classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = train_df['severity'].nunique()

model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss functions and optimizer
learining_rate = 0.0001 
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learining_rate)
num_epochs = 50
# Early stopping parameters
# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []

# Early stopping parameters
diverge_count = 0              # Counter for divergence-based stopping
stop_threshold = 0.05           # Threshold for divergence
max_diverge_count = 5         # Max number of epochs with diverging validation loss

patience_counter = 0           # Counter for plateau-based stopping    
patience = 6                 # Number of epochs for validation loss plateau

 
 

# Calculate number of layers in the model
num_layers = len(list(model.children()))


In [ ]:
import torch
import torch.optim as optim
import os
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt

# Assuming the model, criterion, and optimizer are already defined
# Assuming `train_loader` and `val_loader` are the DataLoader for training and validation sets

# Define the criterion
criterion = nn.MSELoss()

# Initialize variables to keep track of the best validation loss
best_val_loss = float('inf')
best_model_path = "best_model_weights_epoch_1.pt"


# Adjust the model's output layer to match the target tensor size
model.fc = nn.Linear(model.fc.in_features, 2)  # Output x and y coordinates

# MLflow experiment setup
experiment_name = "MRI_Localization_ResNet50"
mlflow.set_experiment(experiment_name)

# Training loop with MLflow tracking
if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run():
    mlflow.log_param("learning_rate", learining_rate)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("model_architecture", "ResNet-50 for Localization")
    mlflow.log_param("output_coordinates", "x, y")

    # Set descriptive tags for the model
    mlflow.set_tag("model_description", "ResNet-50 for 3 classification and Sagittal T2/STIR and Sagittal T1 images")

    # Example input tensor with the same shape as the model's expected input
    example_input = torch.randn(1, 3, 224, 224)  # Batch size 1, 3 color channels, 224x224 image


    # Initialize lists to store loss values for plotting
    train_losses = []
    val_losses = []

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        train_loss = 0.0
        
        # Training phase
        for images, coordinates in train_loader:
            images, coordinates = images.to(device), coordinates.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, coordinates)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)  # Average training loss for the epoch
        train_losses.append(train_loss)  # Store train loss for plotting

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():  # Disable gradient tracking for validation
            for images, coordinates in val_loader:
                images, coordinates = images.to(device), coordinates.to(device)
                outputs = model(images)
                loss = criterion(outputs, coordinates)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)  # Average validation loss for the epoch
        val_losses.append(val_loss)  # Store validation loss for plotting

        # Log losses to MLflow
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Save the model at the current epoch
        model_weight_path = f"epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), model_weight_path)
        mlflow.log_artifact(model_weight_path)  # Log the model weights artifact
        os.remove(model_weight_path)  # Optionally delete local file after logging

        # Check if validation loss improves and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # Reset patience counter if validation loss improves
            best_model_path = f"best_model_weights_epoch_{epoch + 1}.pt"
            torch.save(model.state_dict(), best_model_path)
            mlflow.log_artifact(best_model_path)  # Log the best model weights
            os.remove(best_model_path)  # Optionally, delete the local file after logging
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1} due to lack of validation loss improvement.")
                break

        # Early stopping based on validation loss divergence (optional)
        if val_loss > best_val_loss * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch + 1} due to validation loss divergence.")
                break
        else:
            diverge_count = 0  # Reset diverge count if validation loss doesn't diverge


        # Log final model
    example_input_np = example_input.numpy()
    mlflow.pytorch.log_model(model, "final_model", input_example=example_input_np)
        

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(range(num_epochs), train_losses, label='Train Loss')
    plt.plot(range(num_epochs), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.savefig("loss_curve.png")
    mlflow.log_artifact("loss_curve.png")  # Log the loss curve image


Epoch [1/50], Train Loss: 84235.7098, Validation Loss: 83681.1234
Epoch [2/50], Train Loss: 83023.2943, Validation Loss: 81403.3736
Epoch [3/50], Train Loss: 82614.4420, Validation Loss: 84004.3596
Epoch [4/50], Train Loss: 82262.2913, Validation Loss: 82896.3764


In [166]:
sssss

NameError: name 'sssss' is not defined

test predicion

In [167]:

test_df = test_df.drop(['severity', 'condition', 'level', 'series_id', 'missing_image'], axis=1)


In [168]:
print(test_df.to_string(index=False, header=True)) 

  study_id                                            image_path
2775207739  data/train_images_origin/2775207739/3249541180/8.dcm
 664153360  data/train_images_origin/664153360/1076245514/13.dcm
2273432465   data/train_images_origin/2273432465/114306451/9.dcm
3777149998  data/train_images_origin/3777149998/3550597941/8.dcm
3967802493 data/train_images_origin/3967802493/2054070341/11.dcm
2780118855 data/train_images_origin/2780118855/2064060968/14.dcm
 247968996   data/train_images_origin/247968996/256361821/14.dcm
 957176622   data/train_images_origin/957176622/1889022706/1.dcm
4075603869  data/train_images_origin/4075603869/2921580289/3.dcm
3605654232    data/train_images_origin/3605654232/26446529/3.dcm
1839242409 data/train_images_origin/1839242409/3211434109/13.dcm
 779162887  data/train_images_origin/779162887/2596438821/18.dcm
 886995462  data/train_images_origin/886995462/3051597267/16.dcm
2348702073   data/train_images_origin/2348702073/323466396/5.dcm
3550756125  data/train_im

predicting

In [171]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import mlflow
import mlflow.pytorch
from PIL import Image
import pydicom  # Import pydicom to handle DICOM files

# Step 1: Define the transformation to be applied to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the expected input size of the model
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB (3 channels)
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Step 2: Define a custom Dataset to load images from the DataFrame
class MRILocalizationDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image_path']
        dicom_image = pydicom.dcmread(image_path)  # Read the DICOM file
        image_array = dicom_image.pixel_array.astype('float32')  # Convert the DICOM pixel data to float32

        image = Image.fromarray(image_array)  # Convert NumPy array to PIL Image

        if self.transform:
            image = self.transform(image)  # Apply transformations (including ToTensor)

        return image, image_path


# Replace <run_id> with your actual run ID and <path_to_model> with the artifact path used during logging
model_path = r"C:\Users\HP1\Desktop\Spiced\capstone-project\mlruns\357139645848513089\1c2325d59a33452aab3a227c92533df9\artifacts\final_model"
loaded_model = mlflow.pytorch.load_model(model_path)

loaded_model.eval()  # Set the model to evaluation mode

# Check if CUDA is available and use GPU if possible, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ensure the model is moved to the right device
loaded_model = loaded_model.to(device)


# Step 5: Define a function to localize images
def localize_images(model, test_df, transform):
    dataset = MRILocalizationDataset(test_df, transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)  # Load one image at a time

    predictions = []
    
    with torch.no_grad():  # No need to track gradients during inference
        for images, image_paths in dataloader:
            images = images.to(device)  # Move image to the correct device (GPU/CPU)
            outputs = model(images)  # Get the model's predictions (x, y coordinates)
            predicted_x, predicted_y = outputs[0]  # Assuming model outputs a tuple (x, y)
            predictions.append((predicted_x.item(), predicted_y.item(), image_paths[0]))

    return predictions

# Step 6: Get predictions for new images
predictions = localize_images(loaded_model, test_df, transform)

# Step 7: Print predictions
for idx, (x, y, image_path) in enumerate(predictions):
    print(f"Image {idx+1}: {image_path} - Predicted coordinates (x, y) = ({x:.2f}, {y:.2f})")


Image 1: data/train_images_origin/2775207739/3249541180/8.dcm - Predicted coordinates (x, y) = (0.62, 0.72)
Image 2: data/train_images_origin/664153360/1076245514/13.dcm - Predicted coordinates (x, y) = (3.67, 5.86)
Image 3: data/train_images_origin/2273432465/114306451/9.dcm - Predicted coordinates (x, y) = (1.57, 2.37)
Image 4: data/train_images_origin/3777149998/3550597941/8.dcm - Predicted coordinates (x, y) = (1.95, 3.06)
Image 5: data/train_images_origin/3967802493/2054070341/11.dcm - Predicted coordinates (x, y) = (4.32, 7.06)
Image 6: data/train_images_origin/2780118855/2064060968/14.dcm - Predicted coordinates (x, y) = (0.23, -0.08)
Image 7: data/train_images_origin/247968996/256361821/14.dcm - Predicted coordinates (x, y) = (8.64, 14.41)
Image 8: data/train_images_origin/957176622/1889022706/1.dcm - Predicted coordinates (x, y) = (9.88, 16.94)
Image 9: data/train_images_origin/4075603869/2921580289/3.dcm - Predicted coordinates (x, y) = (4.36, 6.97)
Image 10: data/train_image

In [194]:
test_data = test_df

In [195]:
# calculate prbabilities and probability list
# calculate prbabilities and probability list

# FILEPATH: /c:/Users/HP1/Desktop/Spiced/capstone-project/_5_training_localization_3_cat.ipynb
test_dataset = MRILocalizationDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)

        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



Using device: cpu


KeyboardInterrupt: 

In [181]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]
 

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)
 
# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

,study_id,image_path,Probability_Class_0,Probability_Class_1,Predicted_Class
6296,2775207739,data/train_images_origin/2775207739/3249541180...,0.4744,0.5256,1
1496,664153360,data/train_images_origin/664153360/1076245514/...,0.1012,0.8988,1
5040,2273432465,data/train_images_origin/2273432465/114306451/...,0.3093,0.6907,1
8486,3777149998,data/train_images_origin/3777149998/3550597941...,0.2470,0.7530,1
8959,3967802493,data/train_images_origin/3967802493/2054070341...,0.0604,0.9396,1


In [191]:
from sklearn.metrics import confusion_matrix
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df[['Probability_Class_0', 'Probability_Class_1']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')


KeyError: 'severity'